In [4]:
pip install pandas numpy matplotlib seaborn openpyxl

Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load data
df_analysis = pd.read_csv('cleaned_chronic_absence_data.csv')

# Set Times New Roman (will work in VS Code)
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 11

In [28]:
import plotly.graph_objects as go
import numpy as np

# INTERACTIVE VISUALIZATION 1: DISTRIBUTION OF ATTENDANCE RATES
california_gold = '#FDB515'

# Create bins manually for full control
bins = np.linspace(df_analysis['AttRate'].min(), df_analysis['AttRate'].max(), 51)
hist, bin_edges = np.histogram(df_analysis['AttRate'], bins=bins)
bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2

# Assign colors based on 90% threshold
colors = [california_gold if x >= 0.90 else 'darkred' for x in bin_centers]

# Create the figure
fig = go.Figure()

# Add bars
fig.add_trace(go.Bar(
    x=bin_centers,
    y=hist,
    marker_color=colors,
    width=bin_edges[1] - bin_edges[0],
    hovertemplate='Attendance Rate: %{x:.1%}<br>Students: %{y:,}<extra></extra>',
    showlegend=False
))

# Add mean line (green)
fig.add_vline(
    x=df_analysis['AttRate'].mean(),
    line_dash="dash",
    line_color="green",
    line_width=3,
    annotation_text=f"Mean: {df_analysis['AttRate'].mean():.1%}",
    annotation_position="top left",
    annotation_font={'family': 'Times New Roman', 'size': 12, 'color': 'green'}
)

# Add median line (lime green)
fig.add_vline(
    x=df_analysis['AttRate'].median(),
    line_dash="dash",
    line_color="limegreen",
    line_width=3,
    annotation_text=f"Median: {df_analysis['AttRate'].median():.1%}",
    annotation_position="top right",
    annotation_font={'family': 'Times New Roman', 'size': 12, 'color': 'limegreen'}
)

# Update layout with Times New Roman
fig.update_layout(
    title={
        'text': 'Distribution of Attendance Rates Among Chronically Absent Students',
        'font': {'family': 'Times New Roman', 'size': 18, 'color': 'black'},
        'x': 0.5,
        'xanchor': 'center'
    },
    xaxis_title={
        'text': 'Attendance Rate',
        'font': {'family': 'Times New Roman', 'size': 14}
    },
    yaxis_title={
        'text': 'Number of Students',
        'font': {'family': 'Times New Roman', 'size': 14}
    },
    font={'family': 'Times New Roman', 'size': 12},
    width=1000,
    height=600,
    plot_bgcolor='white',
    xaxis={'showgrid': False, 'zeroline': False},
    yaxis={'showgrid': False, 'zeroline': False},
    bargap=0,
    hovermode='closest'
)

# Show the plot
fig.show()

# Optional: Save as HTML for sharing
fig.write_html('attendance_distribution_interactive.html')

print("\n📊 KEY INSIGHT:")
print("   Red bars = Students below 90% (chronically absent)")
print("   Gold bars = Students at/above 90% (at risk but not chronic)")
print(f"   {(df_analysis['AttRate'] < 0.90).sum():,} students ({(df_analysis['AttRate'] < 0.90).sum()/len(df_analysis)*100:.1f}%) are chronically absent (<90%)")# After each fig.show(), save as HTML
fig.write_html('01_attendance_distribution.html')


print("✓ All HTML files saved!")


📊 KEY INSIGHT:
   Red bars = Students below 90% (chronically absent)
   Gold bars = Students at/above 90% (at risk but not chronic)
   9,099 students (51.5%) are chronically absent (<90%)
✓ All HTML files saved!


In [9]:
# VISUALIZATION 2: STUDENTS BY SEVERITY CATEGORY (VIRIDIS COLORS)
print("Creating Visualization 2: Students by Severity Category")
print("="*70)

import plotly.express as px

severity_order = ['Severe', 'High Chronic', 'Moderate Chronic', 'Low Risk']
severity_counts = df_analysis['Absence_Severity'].value_counts()

# Use Viridis color scale
viridis_colors = px.colors.sequential.Viridis
# Select 4 colors evenly spaced from Viridis
colors = [viridis_colors[0], viridis_colors[3], viridis_colors[6], viridis_colors[9]]

fig = go.Figure()

# Add each bar
for i, severity in enumerate(severity_order):
    count = severity_counts[severity]
    fig.add_trace(go.Bar(
        x=[severity],
        y=[count],
        marker_color=colors[i],
        text=[f'{count:,}'],
        textposition='outside',
        textfont={'family': 'Times New Roman', 'size': 13, 'color': 'black'},
        hovertemplate=f'<b>{severity}</b><br>Students: %{{y:,}}<extra></extra>',
        showlegend=False,
        width=0.6
    ))

fig.update_layout(
    title={
        'text': 'Students by Absence Severity Category',
        'font': {'family': 'Times New Roman', 'size': 18, 'color': 'black'},
        'x': 0.5,
        'xanchor': 'center'
    },
    xaxis_title={
        'text': 'Absence Severity Category',
        'font': {'family': 'Times New Roman', 'size': 14}
    },
    yaxis_title={
        'text': 'Number of Students',
        'font': {'family': 'Times New Roman', 'size': 14}
    },
    font={'family': 'Times New Roman', 'size': 12},
    width=1000,
    height=600,
    plot_bgcolor='white',
    xaxis={'showgrid': False},
    yaxis={'showgrid': False},
    hovermode='closest'
)

fig.show()

# Optional: Save as HTML
fig.write_html('severity_category_viridis.html')

print("\n📊 KEY INSIGHT:")
print(f"   Severe (<70%):          {severity_counts['Severe']:,} students - Crisis intervention")
print(f"   High Chronic (70-79%):  {severity_counts['High Chronic']:,} students - Intensive support")
print(f"   Moderate Chronic (80-89%): {severity_counts['Moderate Chronic']:,} students - Active monitoring")
print(f"   Low Risk (90-100%):     {severity_counts['Low Risk']:,} students - Minimal intervention")

Creating Visualization 2: Students by Severity Category



📊 KEY INSIGHT:
   Severe (<70%):          1,766 students - Crisis intervention
   High Chronic (70-79%):  1,851 students - Intensive support
   Moderate Chronic (80-89%): 6,043 students - Active monitoring
   Low Risk (90-100%):     7,996 students - Minimal intervention


In [11]:
# VISUALIZATION 3: ATTENDANCE BY SCHOOL LEVEL (INTERACTIVE)
print("Creating Visualization 3: Attendance by School Level")
print("="*70)

school_order = ['Elementary', 'Middle', 'High School', 'Early Childhood', 'Post-Secondary']
school_means = df_analysis.groupby('School_Level')['AttRate'].mean()
school_counts = df_analysis.groupby('School_Level').size()

# Use Viridis colors
viridis_colors = px.colors.sequential.Viridis
colors = [viridis_colors[i] for i in [0, 3, 6, 8, 9]]

fig = go.Figure()

# Add bars
for i, level in enumerate(school_order):
    mean_val = school_means[level]
    count = school_counts[level]
    fig.add_trace(go.Bar(
        x=[level],
        y=[mean_val],
        marker_color=colors[i],
        text=[f'{mean_val:.1%}'],
        textposition='outside',
        textfont={'family': 'Times New Roman', 'size': 13, 'color': 'black'},
        hovertemplate=f'<b>{level}</b><br>Mean Attendance: %{{y:.1%}}<br>Students: {count:,}<extra></extra>',
        showlegend=False,
        width=0.6
    ))

# Add 90% chronic threshold line
fig.add_hline(
    y=0.90,
    line_dash="dash",
    line_color="red",
    line_width=2.5,
    annotation_text="90% Chronic Threshold",
    annotation_position="right",
    annotation_font={'family': 'Times New Roman', 'size': 11, 'color': 'red'}
)

fig.update_layout(
    title={
        'text': 'Mean Attendance Rate by School Level',
        'font': {'family': 'Times New Roman', 'size': 18, 'color': 'black'},
        'x': 0.5,
        'xanchor': 'center'
    },
    xaxis_title={
        'text': 'School Level',
        'font': {'family': 'Times New Roman', 'size': 14}
    },
    yaxis_title={
        'text': 'Mean Attendance Rate',
        'font': {'family': 'Times New Roman', 'size': 14}
    },
    font={'family': 'Times New Roman', 'size': 12},
    width=1000,
    height=600,
    plot_bgcolor='white',
    xaxis={'showgrid': False},
    yaxis={'showgrid': False, 'range': [0.75, 0.95]},
    hovermode='closest'
)

fig.show()

# Optional: Save as HTML
fig.write_html('school_level_attendance.html')

print("\n📊 KEY INSIGHT:")
print(f"   Elementary:      {school_means['Elementary']:.1%} (n={school_counts['Elementary']:,}) - BEST")
print(f"   Middle:          {school_means['Middle']:.1%} (n={school_counts['Middle']:,})")
print(f"   High School:     {school_means['High School']:.1%} (n={school_counts['High School']:,}) - WORST")
print(f"   Early Childhood: {school_means['Early Childhood']:.1%} (n={school_counts['Early Childhood']:,})")
print(f"   Post-Secondary:  {school_means['Post-Secondary']:.1%} (n={school_counts['Post-Secondary']:,})")
print(f"\n🚩 Gap between Elementary and High School: {(school_means['Elementary'] - school_means['High School']):.1%}")

Creating Visualization 3: Attendance by School Level



📊 KEY INSIGHT:
   Elementary:      87.2% (n=8,131) - BEST
   Middle:          85.4% (n=3,484)
   High School:     79.9% (n=4,967) - WORST
   Early Childhood: 85.0% (n=980)
   Post-Secondary:  80.1% (n=94)

🚩 Gap between Elementary and High School: 7.3%


In [12]:
# VISUALIZATION 3: ATTENDANCE BY SCHOOL LEVEL (WITH GAP ANNOTATION)
print("Creating Visualization 3: Attendance by School Level")
print("="*70)

school_order = ['Elementary', 'Middle', 'High School', 'Early Childhood', 'Post-Secondary']
school_means = df_analysis.groupby('School_Level')['AttRate'].mean()
school_counts = df_analysis.groupby('School_Level').size()

# Use Viridis colors
viridis_colors = px.colors.sequential.Viridis
colors = [viridis_colors[i] for i in [0, 3, 6, 8, 9]]

fig = go.Figure()

# Add bars
for i, level in enumerate(school_order):
    mean_val = school_means[level]
    count = school_counts[level]
    fig.add_trace(go.Bar(
        x=[level],
        y=[mean_val],
        marker_color=colors[i],
        text=[f'{mean_val:.1%}'],
        textposition='outside',
        textfont={'family': 'Times New Roman', 'size': 13, 'color': 'black'},
        hovertemplate=f'<b>{level}</b><br>Mean Attendance: %{{y:.1%}}<br>Students: {count:,}<extra></extra>',
        showlegend=False,
        width=0.6
    ))

# Add 90% chronic threshold line
fig.add_hline(
    y=0.90,
    line_dash="dash",
    line_color="red",
    line_width=2.5,
    annotation_text="90% Chronic Threshold",
    annotation_position="right",
    annotation_font={'family': 'Times New Roman', 'size': 11, 'color': 'red'}
)

# Add gap annotation between Elementary and High School
gap = school_means['Elementary'] - school_means['High School']
fig.add_annotation(
    x='Elementary',
    y=school_means['Elementary'],
    ax='High School',
    ay=school_means['High School'],
    xref='x',
    yref='y',
    axref='x',
    ayref='y',
    showarrow=True,
    arrowhead=3,
    arrowsize=1.5,
    arrowwidth=2,
    arrowcolor='darkred',
    text=f'Gap: {gap:.1%}',
    font={'family': 'Times New Roman', 'size': 13, 'color': 'darkred', 'weight': 'bold'},
    bgcolor='white',
    bordercolor='darkred',
    borderwidth=2,
    borderpad=4
)

fig.update_layout(
    title={
        'text': 'Mean Attendance Rate by School Level',
        'font': {'family': 'Times New Roman', 'size': 18, 'color': 'black'},
        'x': 0.5,
        'xanchor': 'center'
    },
    xaxis_title={
        'text': 'School Level',
        'font': {'family': 'Times New Roman', 'size': 14}
    },
    yaxis_title={
        'text': 'Mean Attendance Rate',
        'font': {'family': 'Times New Roman', 'size': 14}
    },
    font={'family': 'Times New Roman', 'size': 12},
    width=1000,
    height=600,
    plot_bgcolor='white',
    xaxis={'showgrid': False},
    yaxis={'showgrid': False, 'range': [0.75, 0.95]},
    hovermode='closest'
)

fig.show()

# Optional: Save as HTML
fig.write_html('school_level_attendance.html')

print("\n📊 KEY INSIGHT:")
print(f"   Elementary:      {school_means['Elementary']:.1%} (n={school_counts['Elementary']:,}) - BEST")
print(f"   Middle:          {school_means['Middle']:.1%} (n={school_counts['Middle']:,})")
print(f"   High School:     {school_means['High School']:.1%} (n={school_counts['High School']:,}) - WORST")
print(f"\n🚩 Gap between Elementary and High School: {gap:.1%}")

Creating Visualization 3: Attendance by School Level



📊 KEY INSIGHT:
   Elementary:      87.2% (n=8,131) - BEST
   Middle:          85.4% (n=3,484)
   High School:     79.9% (n=4,967) - WORST

🚩 Gap between Elementary and High School: 7.3%


In [15]:
# VISUALIZATION 4: ATTENDANCE BY ETHNICITY (WITH EQUITY GAP)
print("Creating Visualization 4: Attendance by Ethnicity")
print("="*70)

eth_means = df_analysis.groupby('Eth')['AttRate'].mean().sort_values()
eth_counts = df_analysis.groupby('Eth').size()

# Use Viridis colors
viridis_colors = px.colors.sequential.Viridis
n_ethnicities = len(eth_means)
colors = [viridis_colors[int(i * (len(viridis_colors)-1) / (n_ethnicities-1))] 
          for i in range(n_ethnicities)]

fig = go.Figure()

# Add horizontal bars
for i, (ethnicity, mean_val) in enumerate(eth_means.items()):
    count = eth_counts[ethnicity]
    fig.add_trace(go.Bar(
        y=[ethnicity],
        x=[mean_val],
        orientation='h',
        marker_color=colors[i],
        text=[f'{mean_val:.1%}'],
        textposition='outside',
        textfont={'family': 'Times New Roman', 'size': 12, 'color': 'black'},
        hovertemplate=f'<b>{ethnicity}</b><br>Mean Attendance: %{{x:.1%}}<br>Students: {count:,}<extra></extra>',
        showlegend=False
    ))

# Add 90% chronic threshold line
fig.add_vline(
    x=0.90,
    line_dash="dash",
    line_color="red",
    line_width=2.5,
    annotation_text="90% Threshold",
    annotation_position="top",
    annotation_font={'family': 'Times New Roman', 'size': 11, 'color': 'red'}
)

# Add equity gap annotation
lowest_eth = eth_means.index[0]
highest_eth = eth_means.index[-1]
gap = eth_means[highest_eth] - eth_means[lowest_eth]

fig.add_annotation(
    x=eth_means[lowest_eth],
    y=lowest_eth,
    ax=eth_means[highest_eth],
    ay=highest_eth,
    xref='x',
    yref='y',
    axref='x',
    ayref='y',
    showarrow=True,
    arrowhead=3,
    arrowsize=1.5,
    arrowwidth=2,
    arrowcolor='darkred',
    text=f'Equity Gap: {gap:.1%}',
    font={'family': 'Times New Roman', 'size': 13, 'color': 'darkred', 'weight': 'bold'},
    bgcolor='white',
    bordercolor='darkred',
    borderwidth=2,
    borderpad=4
)

fig.update_layout(
    title={
        'text': 'Mean Attendance Rate by Ethnicity',
        'font': {'family': 'Times New Roman', 'size': 18, 'color': 'black'},
        'x': 0.5,
        'xanchor': 'center'
    },
    xaxis_title={
        'text': 'Mean Attendance Rate',
        'font': {'family': 'Times New Roman', 'size': 14}
    },
    yaxis_title={
        'text': 'Ethnicity',
        'font': {'family': 'Times New Roman', 'size': 14}
    },
    font={'family': 'Times New Roman', 'size': 12},
    width=1000,
    height=700,
    plot_bgcolor='white',
    xaxis={'showgrid': False, 'range': [0.75, 0.95]},
    yaxis={'showgrid': False},
    hovermode='closest'
)

fig.show()

# Optional: Save as HTML
fig.write_html('ethnicity_attendance.html')

print("\n📊 KEY INSIGHT - RACIAL EQUITY GAPS:")
print(f"   Lowest:  Pacific Islander     {eth_means['Pacific Islander']:.1%} (n={eth_counts['Pacific Islander']:,})")
print(f"   Highest: White                {eth_means['White']:.1%} (n={eth_counts['White']:,})")
print(f"   🚩 Equity Gap: {gap:.1%} difference between White and Pacific Islander students")
print(f"\n   Latino students: {eth_means['Latino']:.1%} (n={eth_counts['Latino']:,}) - 53% of dataset")
print(f"   African American: {eth_means['African American']:.1%} (n={eth_counts['African American']:,}) - 23% of dataset")
print(f"   Combined Latino + African American = 76% of chronically absent students")

Creating Visualization 4: Attendance by Ethnicity



📊 KEY INSIGHT - RACIAL EQUITY GAPS:
   Lowest:  Pacific Islander     80.3% (n=210)
   Highest: White                88.1% (n=1,363)
   🚩 Equity Gap: 7.8% difference between White and Pacific Islander students

   Latino students: 84.8% (n=9,419) - 53% of dataset
   African American: 82.2% (n=4,117) - 23% of dataset
   Combined Latino + African American = 76% of chronically absent students


In [16]:
# VISUALIZATION 5: SED STATUS COMPARISON (FULL TERM)
print("Creating Visualization 5: SED Status Comparison")
print("="*70)

sed_means = df_analysis.groupby('SED Status')['AttRate'].mean()
sed_counts = df_analysis['SED Status'].value_counts()

# Viridis colors for contrast
colors = [px.colors.sequential.Viridis[9], px.colors.sequential.Viridis[2]]

fig = go.Figure()

# Create readable labels
labels = {
    'Not SED': 'Not Socioeconomically\nDisadvantaged',
    'SED': 'Socioeconomically\nDisadvantaged'
}

# Add bars
for i, status in enumerate(['Not SED', 'SED']):
    mean_val = sed_means[status]
    count = sed_counts[status]
    fig.add_trace(go.Bar(
        x=[labels[status]],
        y=[mean_val],
        marker_color=colors[i],
        text=[f'{mean_val:.1%}<br>(n={count:,})'],
        textposition='outside',
        textfont={'family': 'Times New Roman', 'size': 13, 'color': 'black'},
        hovertemplate=f'<b>{labels[status].replace("<br>", " ")}</b><br>Mean Attendance: %{{y:.1%}}<br>Students: {count:,}<extra></extra>',
        showlegend=False,
        width=0.5
    ))

# Add 90% chronic threshold line
fig.add_hline(
    y=0.90,
    line_dash="dash",
    line_color="red",
    line_width=2.5,
    annotation_text="90% Chronic Threshold",
    annotation_position="right",
    annotation_font={'family': 'Times New Roman', 'size': 11, 'color': 'red'}
)

# Add equity gap annotation
gap = sed_means['Not SED'] - sed_means['SED']
fig.add_annotation(
    x=labels['Not SED'],
    y=sed_means['Not SED'],
    ax=labels['SED'],
    ay=sed_means['SED'],
    xref='x',
    yref='y',
    axref='x',
    ayref='y',
    showarrow=True,
    arrowhead=3,
    arrowsize=1.5,
    arrowwidth=2,
    arrowcolor='darkred',
    text=f'Equity Gap: {gap:.1%}',
    font={'family': 'Times New Roman', 'size': 14, 'color': 'darkred', 'weight': 'bold'},
    bgcolor='white',
    bordercolor='darkred',
    borderwidth=2,
    borderpad=4
)

fig.update_layout(
    title={
        'text': 'Attendance Rate by Socioeconomic Status',
        'font': {'family': 'Times New Roman', 'size': 18, 'color': 'black'},
        'x': 0.5,
        'xanchor': 'center'
    },
    xaxis_title={
        'text': 'Socioeconomic Status',
        'font': {'family': 'Times New Roman', 'size': 14}
    },
    yaxis_title={
        'text': 'Mean Attendance Rate',
        'font': {'family': 'Times New Roman', 'size': 14}
    },
    font={'family': 'Times New Roman', 'size': 12},
    width=900,
    height=600,
    plot_bgcolor='white',
    xaxis={'showgrid': False},
    yaxis={'showgrid': False, 'range': [0.75, 0.95]},
    hovermode='closest'
)

fig.show()

# Optional: Save as HTML
fig.write_html('sed_status_comparison.html')

print("\n📊 KEY INSIGHT - SOCIOECONOMIC EQUITY GAP:")
print(f"   Not Socioeconomically Disadvantaged: {sed_means['Not SED']:.1%} (n={sed_counts['Not SED']:,})")
print(f"   Socioeconomically Disadvantaged:     {sed_means['SED']:.1%} (n={sed_counts['SED']:,})")
print(f"   🚩 Equity Gap: {gap:.1%} lower attendance for SED students")
print(f"   📊 {(sed_counts['SED']/len(df_analysis)*100):.1f}% of chronically absent students are SED")

Creating Visualization 5: SED Status Comparison



📊 KEY INSIGHT - SOCIOECONOMIC EQUITY GAP:
   Not Socioeconomically Disadvantaged: 90.1% (n=1,845)
   Socioeconomically Disadvantaged:     84.0% (n=15,811)
   🚩 Equity Gap: 6.1% lower attendance for SED students
   📊 89.6% of chronically absent students are SED


In [17]:
# VISUALIZATION 6: SUSPENSION IMPACT (INTERACTIVE)
print("Creating Visualization 6: Suspension Impact")
print("="*70)

suspended = df_analysis[df_analysis['NumSusp'] > 0]
not_suspended = df_analysis[df_analysis['NumSusp'] == 0]

susp_means = [not_suspended['AttRate'].mean(), suspended['AttRate'].mean()]
susp_counts = [len(not_suspended), len(suspended)]

# Viridis colors for contrast
colors = [px.colors.sequential.Viridis[8], px.colors.sequential.Viridis[1]]

fig = go.Figure()

labels = ['Not Suspended', 'Suspended']

# Add bars
for i, label in enumerate(labels):
    mean_val = susp_means[i]
    count = susp_counts[i]
    fig.add_trace(go.Bar(
        x=[label],
        y=[mean_val],
        marker_color=colors[i],
        text=[f'{mean_val:.1%}<br>(n={count:,})'],
        textposition='outside',
        textfont={'family': 'Times New Roman', 'size': 13, 'color': 'black'},
        hovertemplate=f'<b>{label}</b><br>Mean Attendance: %{{y:.1%}}<br>Students: {count:,}<extra></extra>',
        showlegend=False,
        width=0.5
    ))

# Add 90% chronic threshold line
fig.add_hline(
    y=0.90,
    line_dash="dash",
    line_color="red",
    line_width=2.5,
    annotation_text="90% Chronic Threshold",
    annotation_position="right",
    annotation_font={'family': 'Times New Roman', 'size': 11, 'color': 'red'}
)

# Add gap annotation
gap = susp_means[0] - susp_means[1]
fig.add_annotation(
    x=labels[0],
    y=susp_means[0],
    ax=labels[1],
    ay=susp_means[1],
    xref='x',
    yref='y',
    axref='x',
    ayref='y',
    showarrow=True,
    arrowhead=3,
    arrowsize=1.5,
    arrowwidth=2,
    arrowcolor='darkred',
    text=f'Impact: {gap:.1%}<br>(p < 0.001)',
    font={'family': 'Times New Roman', 'size': 14, 'color': 'darkred', 'weight': 'bold'},
    bgcolor='white',
    bordercolor='darkred',
    borderwidth=2,
    borderpad=4
)

fig.update_layout(
    title={
        'text': 'Attendance Rate: Suspended vs Not Suspended Students',
        'font': {'family': 'Times New Roman', 'size': 18, 'color': 'black'},
        'x': 0.5,
        'xanchor': 'center'
    },
    xaxis_title={
        'text': 'Suspension Status',
        'font': {'family': 'Times New Roman', 'size': 14}
    },
    yaxis_title={
        'text': 'Mean Attendance Rate',
        'font': {'family': 'Times New Roman', 'size': 14}
    },
    font={'family': 'Times New Roman', 'size': 12},
    width=900,
    height=600,
    plot_bgcolor='white',
    xaxis={'showgrid': False},
    yaxis={'showgrid': False, 'range': [0.75, 0.95]},
    hovermode='closest'
)

fig.show()

# Optional: Save as HTML
fig.write_html('suspension_impact.html')

print("\n📊 KEY INSIGHT - SUSPENSION & ATTENDANCE:")
print(f"   Not Suspended: {susp_means[0]:.1%} (n={susp_counts[0]:,}) - 95% of students")
print(f"   Suspended:     {susp_means[1]:.1%} (n={susp_counts[1]:,}) - 5% of students")
print(f"   ⚠️ Impact: {gap:.1%} lower attendance for suspended students")
print(f"   📊 Statistically significant (t-test: p < 0.001)")

Creating Visualization 6: Suspension Impact



📊 KEY INSIGHT - SUSPENSION & ATTENDANCE:
   Not Suspended: 84.9% (n=16,779) - 95% of students
   Suspended:     80.5% (n=877) - 5% of students
   ⚠️ Impact: 4.4% lower attendance for suspended students
   📊 Statistically significant (t-test: p < 0.001)


In [18]:
# VISUALIZATION 7: BOXPLOT BY SEVERITY CATEGORY (INTERACTIVE)
print("Creating Visualization 7: Distribution by Severity Category")
print("="*70)

severity_order = ['Severe', 'High Chronic', 'Moderate Chronic', 'Low Risk']

# Viridis colors
viridis_colors = px.colors.sequential.Viridis
colors = [viridis_colors[0], viridis_colors[3], viridis_colors[6], viridis_colors[9]]

fig = go.Figure()

# Add box plot for each severity category
for i, severity in enumerate(severity_order):
    data = df_analysis[df_analysis['Absence_Severity'] == severity]['AttRate']
    fig.add_trace(go.Box(
        y=data,
        name=severity,
        marker_color=colors[i],
        boxmean='sd',  # Show mean and standard deviation
        hovertemplate='<b>%{fullData.name}</b><br>Attendance Rate: %{y:.1%}<extra></extra>'
    ))

# Add 90% chronic threshold line
fig.add_hline(
    y=0.90,
    line_dash="dash",
    line_color="red",
    line_width=2.5,
    annotation_text="90% Chronic Threshold",
    annotation_position="right",
    annotation_font={'family': 'Times New Roman', 'size': 11, 'color': 'red'}
)

# Add 80% and 70% lines
fig.add_hline(
    y=0.80,
    line_dash="dot",
    line_color="orange",
    line_width=2,
    annotation_text="80%",
    annotation_position="right",
    annotation_font={'family': 'Times New Roman', 'size': 10, 'color': 'orange'}
)

fig.add_hline(
    y=0.70,
    line_dash="dot",
    line_color="darkred",
    line_width=2,
    annotation_text="70%",
    annotation_position="right",
    annotation_font={'family': 'Times New Roman', 'size': 10, 'color': 'darkred'}
)

fig.update_layout(
    title={
        'text': 'Attendance Rate Distribution by Severity Category',
        'font': {'family': 'Times New Roman', 'size': 18, 'color': 'black'},
        'x': 0.5,
        'xanchor': 'center'
    },
    xaxis_title={
        'text': 'Absence Severity Category',
        'font': {'family': 'Times New Roman', 'size': 14}
    },
    yaxis_title={
        'text': 'Attendance Rate',
        'font': {'family': 'Times New Roman', 'size': 14}
    },
    font={'family': 'Times New Roman', 'size': 12},
    width=1000,
    height=600,
    plot_bgcolor='white',
    xaxis={'showgrid': False},
    yaxis={'showgrid': False},
    showlegend=False,
    hovermode='closest'
)

fig.show()

# Optional: Save as HTML
fig.write_html('severity_boxplot.html')

print("\n📊 KEY INSIGHT - DISTRIBUTION PATTERNS:")
for severity in severity_order:
    data = df_analysis[df_analysis['Absence_Severity'] == severity]['AttRate']
    print(f"   {severity:<20} Median: {data.median():.1%}, Range: {data.min():.1%} - {data.max():.1%}")

Creating Visualization 7: Distribution by Severity Category



📊 KEY INSIGHT - DISTRIBUTION PATTERNS:
   Severe               Median: 56.2%, Range: 0.0% - 70.0%
   High Chronic         Median: 76.1%, Range: 70.1% - 80.0%
   Moderate Chronic     Median: 86.7%, Range: 80.1% - 90.0%
   Low Risk             Median: 92.8%, Range: 90.1% - 95.0%


In [19]:
# Check what location data is available
print("Location columns available:")
print(df_analysis[['AddressResidence', 'CityResidence', 'ZipResidence']].head(10))

print("\nUnique cities:")
print(df_analysis['CityResidence'].value_counts())

print("\nUnique zip codes:")
print(df_analysis['ZipResidence'].value_counts())

Location columns available:
          AddressResidence CityResidence  ZipResidence
0           1931 Myrtle St       Oakland         94607
1             371 98th Ave       Oakland         94603
2  6630 MACARTHUR BLVD #20       Oakland         94605
3            9429 D street       Oakland         94603
4          1915 Auseon Ave       Oakland         94621
5            2540 35th ave       Oakland         94601
6           2524 E 10th St       Oakland         94601
7           4339 Park Blvd       Oakland         94602
8        580 Capell St Apt       Oakland         94610
9            905 106th Ave       Oakland         94603

Unique cities:
CityResidence
Oakland        17151
San Leandro      119
Emeryville        63
Hayward           36
Richmond          34
               ...  
Oakland CA         1
Orinda             1
Suisun City        1
Oaklandd           1
Other              1
Name: count, Length: 64, dtype: int64

Unique zip codes:
ZipResidence
94601    3052
94621    2700
94603   

In [20]:
# VISUALIZATION 9: ATTENDANCE BY OAKLAND REGION (INTERACTIVE)
print("Creating Visualization 9: Attendance by Oakland Region")
print("="*70)

# Define Oakland regions by zip code
# Based on Oakland neighborhood geography
oakland_regions = {
    # West Oakland
    94607: 'West Oakland',
    94608: 'West Oakland',
    94612: 'West Oakland',
    
    # Downtown/Central
    94610: 'Central Oakland',
    94611: 'Central Oakland',
    94618: 'Central Oakland',
    
    # East Oakland
    94601: 'East Oakland',
    94603: 'East Oakland',
    94605: 'East Oakland',
    94621: 'East Oakland',
    94619: 'East Oakland',
    
    # North Oakland
    94609: 'North Oakland',
    94613: 'North Oakland',
    
    # Other
    94602: 'Central Oakland',
    94606: 'East Oakland'
}

# Filter to Oakland only and map regions
oakland_data = df_analysis[df_analysis['CityResidence'] == 'Oakland'].copy()
oakland_data['Region'] = oakland_data['ZipResidence'].map(oakland_regions)

# Remove any unmapped zip codes
oakland_data = oakland_data[oakland_data['Region'].notna()]

print(f"\nTotal Oakland students mapped: {len(oakland_data):,}")
print("\nStudents by region:")
print(oakland_data['Region'].value_counts())

# Calculate statistics by region
region_stats = oakland_data.groupby('Region').agg({
    'AttRate': ['mean', 'median', 'count']
}).round(4)
region_stats.columns = ['Mean', 'Median', 'Count']
region_stats = region_stats.sort_values('Mean')

print("\nAttendance by region:")
print(region_stats)

# Create visualization
fig = go.Figure()

region_order = region_stats.index.tolist()
viridis_colors = px.colors.sequential.Viridis
colors = [viridis_colors[int(i * (len(viridis_colors)-1) / (len(region_order)-1))] 
          for i in range(len(region_order))]

# Add bars
for i, region in enumerate(region_order):
    mean_val = region_stats.loc[region, 'Mean']
    count = int(region_stats.loc[region, 'Count'])
    
    fig.add_trace(go.Bar(
        x=[region],
        y=[mean_val],
        marker_color=colors[i],
        text=[f'{mean_val:.1%}'],
        textposition='outside',
        textfont={'family': 'Times New Roman', 'size': 13, 'color': 'black'},
        hovertemplate=f'<b>{region}</b><br>Mean Attendance: %{{y:.1%}}<br>Students: {count:,}<extra></extra>',
        showlegend=False,
        width=0.6
    ))

# Add 90% chronic threshold line
fig.add_hline(
    y=0.90,
    line_dash="dash",
    line_color="red",
    line_width=2.5,
    annotation_text="90% Chronic Threshold",
    annotation_position="right",
    annotation_font={'family': 'Times New Roman', 'size': 11, 'color': 'red'}
)

# Add geographic equity gap if there are multiple regions
if len(region_order) > 1:
    lowest_region = region_order[0]
    highest_region = region_order[-1]
    gap = region_stats.loc[highest_region, 'Mean'] - region_stats.loc[lowest_region, 'Mean']
    
    fig.add_annotation(
        x=lowest_region,
        y=region_stats.loc[lowest_region, 'Mean'],
        ax=highest_region,
        ay=region_stats.loc[highest_region, 'Mean'],
        xref='x',
        yref='y',
        axref='x',
        ayref='y',
        showarrow=True,
        arrowhead=3,
        arrowsize=1.5,
        arrowwidth=2,
        arrowcolor='darkred',
        text=f'Geographic Gap: {gap:.1%}',
        font={'family': 'Times New Roman', 'size': 14, 'color': 'darkred', 'weight': 'bold'},
        bgcolor='white',
        bordercolor='darkred',
        borderwidth=2,
        borderpad=4
    )

fig.update_layout(
    title={
        'text': 'Mean Attendance Rate by Oakland Region',
        'font': {'family': 'Times New Roman', 'size': 18, 'color': 'black'},
        'x': 0.5,
        'xanchor': 'center'
    },
    xaxis_title={
        'text': 'Oakland Region',
        'font': {'family': 'Times New Roman', 'size': 14}
    },
    yaxis_title={
        'text': 'Mean Attendance Rate',
        'font': {'family': 'Times New Roman', 'size': 14}
    },
    font={'family': 'Times New Roman', 'size': 12},
    width=1000,
    height=600,
    plot_bgcolor='white',
    xaxis={'showgrid': False},
    yaxis={'showgrid': False, 'range': [0.75, 0.95]},
    hovermode='closest'
)

fig.show()

# Optional: Save as HTML
fig.write_html('oakland_region_attendance.html')

print("\n📊 KEY INSIGHT - GEOGRAPHIC EQUITY:")
for region in region_order:
    mean_val = region_stats.loc[region, 'Mean']
    count = int(region_stats.loc[region, 'Count'])
    print(f"   {region:<20} {mean_val:.1%} (n={count:,})")

if len(region_order) > 1:
    print(f"\n🚩 Geographic Gap: {gap:.1%} between {highest_region} and {lowest_region}")

Creating Visualization 9: Attendance by Oakland Region

Total Oakland students mapped: 17,125

Students by region:
Region
East Oakland       12824
Central Oakland     2104
West Oakland        1598
North Oakland        599
Name: count, dtype: int64

Attendance by region:
                   Mean  Median  Count
Region                                
West Oakland     0.8397  0.8889   1598
East Oakland     0.8426  0.8914  12824
North Oakland    0.8529  0.9000    599
Central Oakland  0.8759  0.9167   2104



📊 KEY INSIGHT - GEOGRAPHIC EQUITY:
   West Oakland         84.0% (n=1,598)
   East Oakland         84.3% (n=12,824)
   North Oakland        85.3% (n=599)
   Central Oakland      87.6% (n=2,104)

🚩 Geographic Gap: 3.6% between Central Oakland and West Oakland


In [21]:
# ANALYSIS: ETHNICITY BY OAKLAND REGION
print("ETHNICITY BY OAKLAND REGION ANALYSIS")
print("="*70)

# Create crosstab of ethnicity by region
ethnic_by_region = pd.crosstab(oakland_data['Region'], oakland_data['Eth'], margins=True)
print("\nStudent counts by ethnicity and region:")
print(ethnic_by_region)

# Calculate percentages by region
ethnic_pct_by_region = pd.crosstab(oakland_data['Region'], oakland_data['Eth'], normalize='index') * 100
print("\n" + "="*70)
print("PERCENTAGE DISTRIBUTION BY REGION")
print("="*70)
print(ethnic_pct_by_region.round(1))

# Key statistics by region
print("\n" + "="*70)
print("ETHNIC COMPOSITION SUMMARY BY REGION")
print("="*70)

for region in ['West Oakland', 'East Oakland', 'North Oakland', 'Central Oakland']:
    if region in oakland_data['Region'].values:
        region_data = oakland_data[oakland_data['Region'] == region]
        total = len(region_data)
        mean_att = region_data['AttRate'].mean()
        
        print(f"\n{region} (Mean Attendance: {mean_att:.1%}, n={total:,}):")
        
        # Top 3 ethnic groups
        top_groups = region_data['Eth'].value_counts().head(3)
        for eth, count in top_groups.items():
            pct = (count / total) * 100
            eth_att = region_data[region_data['Eth'] == eth]['AttRate'].mean()
            print(f"  {eth:<25} {count:>5,} ({pct:>5.1f}%) - Att: {eth_att:.1%}")

# Statistical correlation test
print("\n" + "="*70)
print("CHI-SQUARE TEST: Region vs Ethnicity")
print("="*70)

from scipy.stats import chi2_contingency
contingency = pd.crosstab(oakland_data['Region'], oakland_data['Eth'])
chi2, p_value, dof, expected = chi2_contingency(contingency)

print(f"Chi-square statistic: {chi2:.4f}")
print(f"P-value: {p_value:.4e}")
print(f"Degrees of freedom: {dof}")

if p_value < 0.05:
    print("✓ Statistically significant relationship between region and ethnicity (p < 0.05)")
else:
    print("✗ No statistically significant relationship")

# Key insight: Compare Latino/African American percentages by region
print("\n" + "="*70)
print("LATINO & AFRICAN AMERICAN CONCENTRATION BY REGION")
print("="*70)

for region in region_stats.index:
    region_data = oakland_data[oakland_data['Region'] == region]
    latino_pct = (region_data['Eth'] == 'Latino').sum() / len(region_data) * 100
    aa_pct = (region_data['Eth'] == 'African American').sum() / len(region_data) * 100
    combined = latino_pct + aa_pct
    mean_att = region_data['AttRate'].mean()
    
    print(f"{region:<20} Latino: {latino_pct:>5.1f}%  African American: {aa_pct:>5.1f}%  Combined: {combined:>5.1f}%  |  Att: {mean_att:.1%}")

ETHNICITY BY OAKLAND REGION ANALYSIS

Student counts by ethnicity and region:
Eth              African American  Asian  Filipino  Latino  \
Region                                                       
Central Oakland               403    175        18     619   
East Oakland                 2628    633        45    7971   
North Oakland                 226     16         3     148   
West Oakland                  678    107         4     466   
All                          3935    931        70    9204   

Eth              Multiple Ethnicity  Native American  Not Reported  \
Region                                                               
Central Oakland                 273                8            90   
East Oakland                    497               53           290   
North Oakland                    71                0            25   
West Oakland                     93                3            53   
All                             934               64           458 

In [23]:
# VISUALIZATION 10A: ETHNIC COMPOSITION BY REGION - GROUPED BARS
print("\nCreating Visualization 10: Ethnic Composition by Region (Grouped)")
print("="*70)

# Get top 5 ethnicities for cleaner visualization
top_ethnicities = oakland_data['Eth'].value_counts().head(5).index.tolist()

# Prepare data
region_order_by_att = region_stats.index.tolist()

fig = go.Figure()

# Viridis colors for ethnicities
viridis = px.colors.sequential.Viridis
eth_colors = {eth: viridis[i] for i, eth in enumerate(range(len(top_ethnicities)))}

for i, eth in enumerate(top_ethnicities):
    percentages = []
    for region in region_order_by_att:
        region_subset = oakland_data[oakland_data['Region'] == region]
        pct = (region_subset['Eth'] == eth).sum() / len(region_subset) * 100
        percentages.append(pct)
    
    fig.add_trace(go.Bar(
        name=eth,
        x=region_order_by_att,
        y=percentages,
        marker_color=viridis[int(i * (len(viridis)-1) / (len(top_ethnicities)-1))],
        text=[f'{p:.1f}%' for p in percentages],
        textposition='outside',
        textfont={'family': 'Times New Roman', 'size': 10},
        hovertemplate=f'<b>{eth}</b><br>%{{x}}<br>%{{y:.1f}}%<extra></extra>'
    ))

fig.update_layout(
    barmode='group',
    title={
        'text': 'Ethnic Composition by Oakland Region',
        'font': {'family': 'Times New Roman', 'size': 18, 'color': 'black'},
        'x': 0.5,
        'xanchor': 'center'
    },
    xaxis_title={
        'text': 'Oakland Region (ordered by attendance rate)',
        'font': {'family': 'Times New Roman', 'size': 14}
    },
    yaxis_title={
        'text': 'Percentage of Students',
        'font': {'family': 'Times New Roman', 'size': 14}
    },
    font={'family': 'Times New Roman', 'size': 12},
    width=1100,
    height=600,
    plot_bgcolor='white',
    xaxis={'showgrid': False},
    yaxis={'showgrid': False},
    legend={'font': {'family': 'Times New Roman', 'size': 11}},
    hovermode='closest'
)

fig.show()
fig.write_html('oakland_region_ethnicity_grouped.html')


Creating Visualization 10: Ethnic Composition by Region (Grouped)


In [24]:
# VISUALIZATION 10B: ETHNIC COMPOSITION BY REGION - HEATMAP
print("\nCreating Visualization 10: Ethnic Composition by Region (Heatmap)")
print("="*70)

# Get top ethnicities
top_ethnicities = oakland_data['Eth'].value_counts().head(6).index.tolist()

# Create percentage matrix
region_order_by_att = region_stats.index.tolist()
heatmap_data = []

for eth in top_ethnicities:
    row = []
    for region in region_order_by_att:
        region_subset = oakland_data[oakland_data['Region'] == region]
        pct = (region_subset['Eth'] == eth).sum() / len(region_subset) * 100
        row.append(pct)
    heatmap_data.append(row)

# Create heatmap
fig = go.Figure(data=go.Heatmap(
    z=heatmap_data,
    x=region_order_by_att,
    y=top_ethnicities,
    colorscale='Viridis',
    text=[[f'{val:.1f}%' for val in row] for row in heatmap_data],
    texttemplate='%{text}',
    textfont={'family': 'Times New Roman', 'size': 12, 'color': 'white'},
    hovertemplate='<b>%{y}</b><br>%{x}<br>%{z:.1f}%<extra></extra>',
    colorbar=dict(
        title='Percentage',
        titlefont={'family': 'Times New Roman', 'size': 12},
        tickfont={'family': 'Times New Roman', 'size': 11}
    )
))

fig.update_layout(
    title={
        'text': 'Ethnic Composition by Oakland Region',
        'font': {'family': 'Times New Roman', 'size': 18, 'color': 'black'},
        'x': 0.5,
        'xanchor': 'center'
    },
    xaxis_title={
        'text': 'Oakland Region (ordered by attendance rate)',
        'font': {'family': 'Times New Roman', 'size': 14}
    },
    yaxis_title={
        'text': 'Ethnicity',
        'font': {'family': 'Times New Roman', 'size': 14}
    },
    font={'family': 'Times New Roman', 'size': 12},
    width=1000,
    height=600,
    plot_bgcolor='white',
    xaxis={'showgrid': False},
    yaxis={'showgrid': False}
)

fig.show()
fig.write_html('oakland_region_ethnicity_heatmap.html')

print("\n✅ Visualization complete!")


Creating Visualization 10: Ethnic Composition by Region (Heatmap)



✅ Visualization complete!
